PySpark + Spark SQL Task


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
spark=SparkSession.builder.appName("PracticeProject").enableHiveSupport().getOrCreate()
# Customers Data
customers_data = [
(101, 'Ali', 'ali@gmail.com', 'Mumbai', '2022-05-10'),
(102, 'Neha', 'neha@yahoo.com', 'Delhi', '2023-01-15'),
(103, 'Ravi', 'ravi@hotmail.com', 'Bangalore', '2021-11-01'),
(104, 'Sneha', 'sneha@outlook.com', 'Hyderabad', '2020-07-22'),
(105, 'Amit', 'amit@gmail.com', 'Chennai', '2023-03-10'),
]
orders_data = [
(1, 101, 'Laptop', 'Electronics', 2, 50000.0, '2024-01-10'),
(2, 101, 'Mouse', 'Electronics', 1, 1200.0, '2024-01-15'),
(3, 102, 'Tablet', 'Electronics', 1, 20000.0, '2024-02-01'),
(4, 103, 'Bookshelf', 'Furniture', 1, 3500.0, '2024-02-10'),
(5, 104, 'Mixer', 'Appliances', 1, 5000.0, '2024-02-15'),
(6, 105, 'Notebook', 'Stationery', 5, 500.0, '2024-03-01'),
(7, 102, 'Phone', 'Electronics', 1, 30000.0, '2024-03-02'),
]
customers_df = spark.createDataFrame(customers_data, ["CustomerID", "Name", "Email",
"City", "SignupDate"])
orders_df = spark.createDataFrame(orders_data, ["OrderID", "CustomerID", "Product",
"Category", "Quantity", "Price", "OrderDate"])
spark.sql("CREATE DATABASE IF NOT EXISTS sales")
spark.sql("USE sales")
# Write as tables
customers_df.write.mode("overwrite").saveAsTable("sales.customers")
orders_df.write.mode("overwrite").saveAsTable("sales.orders")

SECTION A: PySpark DataFrame Tasks

In [4]:
# Add a column TotalAmount = Price * Quantity to the orders_df .
orders_df = orders_df.withColumn("TotalAmount", expr("Price * Quantity"))
orders_df.show()

+-------+----------+---------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+---------+-----------+--------+-------+----------+-----------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+---------+-----------+--------+-------+----------+-----------+



In [5]:
# Filter all orders with TotalAmount > 10000 .
orders_df.filter(orders_df.TotalAmount > 10000).show()

+-------+----------+-------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+-------+-----------+--------+-------+----------+-----------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      3|       102| Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      7|       102|  Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+-------+-----------+--------+-------+----------+-----------+



In [6]:
# Standardize the City field in customers_df eg lowercase
customers_df = customers_df.withColumn("City", expr("lower(City)"))
customers_df.show()

+----------+-----+-----------------+---------+----------+
|CustomerID| Name|            Email|     City|SignupDate|
+----------+-----+-----------------+---------+----------+
|       101|  Ali|    ali@gmail.com|   mumbai|2022-05-10|
|       102| Neha|   neha@yahoo.com|    delhi|2023-01-15|
|       103| Ravi| ravi@hotmail.com|bangalore|2021-11-01|
|       104|Sneha|sneha@outlook.com|hyderabad|2020-07-22|
|       105| Amit|   amit@gmail.com|  chennai|2023-03-10|
+----------+-----+-----------------+---------+----------+



In [7]:
# Extract year from OrderDate and add a new column OrderYear .
orders_df = orders_df.withColumn("OrderYear", expr("year(OrderDate)"))
orders_df.show()

+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|     2024|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|     2024|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|     2024|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|     2024|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|     2024|
+-------+----------+---------+-----------+--------+-------+----------+

In [8]:
# Fill null values in any column of your choice with defaults.
orders_df = orders_df.na.fill(0, ["Quantity"])
orders_df.show()

+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|     2024|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|     2024|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|     2024|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|     2024|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|     2024|
+-------+----------+---------+-----------+--------+-------+----------+

In [36]:
# Use when/otherwise to categorize orders:
# <5000 : "Low"
# 5000-20000 : "Medium"
# >20000 : "High"
from pyspark.sql.functions import when, col
orders_df = orders_df.withColumn("OrderCategory",
when(col("TotalAmount") < 5000, "Low").otherwise(when(col("TotalAmount") < 20000, "Medium").otherwise("High")))
orders_df.show()

orders_df.show()

+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+-------------+--------------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|OrderCategory|CleanedProduct|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+-------------+--------------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|         High|        Laptop|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|          Low|         Mouse|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|     2024|         High|        Tablet|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|     2024|          Low|     Bookshelf|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|     2024|       Medium|         Mixer|
|      6|       105| Not

SECTION B: Spark SQL Tasks

In [12]:
# Run a SQL query to list all orders made by “Ali”.
spark.sql("""SELECT o.* FROM sales.orders o
JOIN sales.customers c ON o.CustomerID = c.CustomerID
WHERE c.Name = 'Ali'
""").show()

+-------+----------+-------+-----------+--------+-------+----------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|
+-------+----------+-------+-----------+--------+-------+----------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|
|      2|       101|  Mouse|Electronics|       1| 1200.0|2024-01-15|
+-------+----------+-------+-----------+--------+-------+----------+



In [14]:
# Get total spending by each customer using SQL.
spark.sql("""SELECT c.Name, SUM(o.Price * o.Quantity) AS TotalSpending
FROM sales.orders o
JOIN sales.customers c ON o.CustomerID = c.CustomerID
GROUP BY o.CustomerID, c.Name
""").show()

+-----+-------------+
| Name|TotalSpending|
+-----+-------------+
|  Ali|     101200.0|
| Neha|      50000.0|
| Ravi|       3500.0|
| Amit|       2500.0|
|Sneha|       5000.0|
+-----+-------------+



In [15]:
# Find out which category made the highest total revenue.
spark.sql("""SELECT Category, SUM(Price * Quantity) AS TotalRevenue
FROM sales.orders
GROUP BY Category
ORDER BY TotalRevenue DESC
LIMIT 1
""").show()

+-----------+------------+
|   Category|TotalRevenue|
+-----------+------------+
|Electronics|    151200.0|
+-----------+------------+



In [17]:
# Create a view customer_orders showing CustomerName, Product, TotalAmount .
spark.sql("""CREATE OR REPLACE VIEW sales.customer_orders AS
SELECT c.Name AS CustomerName, o.Product, (o.Price * o.Quantity) as TotalAmount
FROM sales.orders o
JOIN sales.customers c ON o.CustomerID = c.CustomerID
""")

DataFrame[]

In [37]:
# Query the view for products ordered after Feb 2024.
spark.sql("""SELECT * FROM sales.customer_orders
WHERE TotalAmount>0 AND Product IS NOT NULL
AND EXISTS (
    SELECT 1 FROM orders o
    WHERE o.Product = customer_orders.Product
    AND o.OrderDate > '2024-02-01'
)""").show()

+------------+---------+-----------+
|CustomerName|  Product|TotalAmount|
+------------+---------+-----------+
|        Ravi|Bookshelf|     3500.0|
|       Sneha|    Mixer|     5000.0|
|        Amit| Notebook|     2500.0|
|        Neha|    Phone|    30000.0|
+------------+---------+-----------+



SECTION C: Advanced Practice

In [25]:
# Create a Global Temp View from customers_df , then query it using:
# SELECT * FROM global_temp.customers WHERE City = 'Mumbai';
customers_df.createOrReplaceGlobalTempView("customers")
spark.sql("SELECT * FROM global_temp.customers WHERE City = 'mumbai'").show()

+----------+----+-------------+------+----------+
|CustomerID|Name|        Email|  City|SignupDate|
+----------+----+-------------+------+----------+
|       101| Ali|ali@gmail.com|mumbai|2022-05-10|
+----------+----+-------------+------+----------+



In [26]:
# Save the transformed orders_df (with TotalAmount) to a Parquet file.\
orders_df.write.mode("overwrite").parquet("orders_parquet")

In [27]:
# Read back the Parquet file and count how many orders are in it.
parquet_df = spark.read.parquet("orders_parquet")
parquet_df.count()

7

SECTION D: UDF + Built-in Function Tasks

In [28]:
# Write a UDF that masks emails like: ali@gmail.com → a***@gmail.com .
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def mask_email(email):
    if email and '@' in email:
        user, domain = email.split('@')
        return user[0] + '***@' + domain
    return email

mask_email_udf = udf(mask_email, StringType())

customers_df = customers_df.withColumn("MaskedEmail", mask_email_udf("Email"))
customers_df.select("Email", "MaskedEmail").show()


+-----------------+----------------+
|            Email|     MaskedEmail|
+-----------------+----------------+
|    ali@gmail.com|  a***@gmail.com|
|   neha@yahoo.com|  n***@yahoo.com|
| ravi@hotmail.com|r***@hotmail.com|
|sneha@outlook.com|s***@outlook.com|
|   amit@gmail.com|  a***@gmail.com|
+-----------------+----------------+



In [29]:
# Use concat_ws() to create labels like ‘Ali from Mumbai’
from pyspark.sql.functions import concat_ws

labels_df = customers_df.withColumn("Label", concat_ws(" from ", "Name", "City"))
labels_df.select("Name", "City", "Label").show()

+-----+---------+--------------------+
| Name|     City|               Label|
+-----+---------+--------------------+
|  Ali|   mumbai|     Ali from mumbai|
| Neha|    delhi|     Neha from delhi|
| Ravi|bangalore| Ravi from bangalore|
|Sneha|hyderabad|Sneha from hyderabad|
| Amit|  chennai|   Amit from chennai|
+-----+---------+--------------------+



In [30]:
# Use regexp_replace() to remove special characters from Product
from pyspark.sql.functions import regexp_replace

orders_df = orders_df.withColumn("CleanedProduct", regexp_replace("Product", "[^a-zA-Z0-9 ]", ""))
orders_df.select("Product", "CleanedProduct").show()

+---------+--------------+
|  Product|CleanedProduct|
+---------+--------------+
|   Laptop|        Laptop|
|    Mouse|         Mouse|
|   Tablet|        Tablet|
|Bookshelf|     Bookshelf|
|    Mixer|         Mixer|
| Notebook|      Notebook|
|    Phone|         Phone|
+---------+--------------+



In [35]:
#  Calculate Customer Age in Days from SignupDate
from pyspark.sql.functions import datediff, current_date

customers_df = customers_df.withColumn("AgeInDays", datediff(current_date(), "SignupDate"))
customers_df.select("Name", "SignupDate", "AgeInDays").show()

+-----+----------+---------+
| Name|SignupDate|AgeInDays|
+-----+----------+---------+
|  Ali|2022-05-10|     1121|
| Neha|2023-01-15|      871|
| Ravi|2021-11-01|     1311|
|Sneha|2020-07-22|     1778|
| Amit|2023-03-10|      817|
+-----+----------+---------+

